# SSML OSMnx

### Packages

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import osmnx as ox
import pandas as pd
ox.config(use_cache=True, log_console=True)

### Data

In [ ]:
# load GHS UCD database
ghs_full_df = gpd.read_file("GHS/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg")

In [ ]:
# load city dataset
df_cities = pd.read_csv('GHS/world_cities.csv')

## OSMnx Function

In [ ]:
def get_cities_osmnx(df:pd.DataFrame, ghs_df) -> pd.DataFrame:
    '''
    This function returns the GHS DataFrame with 8 urban spatial indicators for all urban centres.
    '''

    df["street_orientation_entropy"] = np.nan
    df["orientation_order_indicator"] = np.nan
    df["weighted_street_orientation_entropy"] = np.nan

    entropy_bins = 36

    counter = 1
    # go through all urban centres in dataset
    for index, row in df.iterrows():
        # reload package for every loop due to errors
        from scipy import stats

        # extract city and country 
        city = str(row['UC_NM_MN'])
        country = str(row['XC_NM_LST'])
        print(f"Currently we are calculating city number {counter}, {city}")

        try:
            # get spatial boundary of relevant city form GHS dataframe
            geom = ghs_df[(ghs_df['UC_NM_MN'] == city) & (ghs_df['XC_NM_LST'] == country)]['geometry']
            # get street network of city
            G = ox.graph_from_polygon(geom.unary_union, network_type='drive')
            # plot street network of city
            fig, ax = ox.plot_graph(G)

            # calculate area of street network
            G_proj = ox.project_graph(G)
            nodes_proj = ox.graph_to_gdfs(G_proj, edges=False)
            graph_area_m = nodes_proj.unary_union.convex_hull.area

            # get edge's bearings of street network
            Gu = ox.add_edge_bearings(ox.get_undirected(G))
            # calculate orientation entropy
            orientation_entropy = ox.bearing.orientation_entropy(Gu)
            # calculate weighted orientation entropy
            weighted_orientation_entropy = ox.bearing.orientation_entropy(Gu, weight='length')

            # calculate orientation order with 36 bins 
            perfect_grid = [1] * 4 + [0] * (entropy_bins - 4)
            min_entropy = stats.entropy(perfect_grid)
            max_entropy = np.log(entropy_bins)    
            orientation_order = 1 - ((orientation_entropy - min_entropy) / (max_entropy - min_entropy)) ** 2

            # get basic statistics of street network
            stats = ox.basic_stats(G_proj, area=graph_area_m, clean_int_tol=True)
            # get other five street network indicators
            median_street_length = stats['street_length_avg']
            average_circuity = stats['circuity_avg']
            average_node_degree = stats['k_avg']
            proportion_nodes_dead_ends = stats['streets_per_node_proportions'][1]
            proportion_nodes_four_intersects = stats['streets_per_node_proportions'][4]

            # add indicators to city dataframe
            df.at[index, "orientation_order_indicator"] = orientation_order
            df.at[index, "street_orientation_entropy"] = orientation_entropy
            df.at[index, "weighted_street_orientation_entropy"] = weighted_orientation_entropy
            df.at[index, "median_street_segment_length"] = median_street_length
            df.at[index, "average_circuity"] = average_circuity
            df.at[index, "average_node_degree"] = average_node_degree
            df.at[index, "proportion_nodes_dead_ends"] = proportion_nodes_dead_ends
            df.at[index, "proportion_nodes_fourway_intersections"] = proportion_nodes_four_intersects

            counter +=1
            
        except:
            # if street network not accessible in OSM, go to following city
            row["street_orientation_entropy"] = 0
            row["orientation_order_indicator"] = 0
            row["weighted_street_orientation_entropy"] = 0
            print(f"We have failed to make the calculations for {city}")

    return df

### Getting Data

In [ ]:
# get street network indicators of cities in city dataset
# WATCH OUT!!! FOR ALL 80 CITIES THIS CAN TAKE ROUGHLY 5 a 6 HOURS!!!
df_cities = get_cities_osmnx(df_cities, ghs_full_df)